In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gym  # Assuming you have a gym-like environment
from gym import spaces
import random
from collections import deque
import matplotlib.pyplot as plt
import io
import imageio.v2 as imageio
import torch.nn.functional as F
from matplotlib.patches import FancyArrow
from shapely.geometry import Point, LineString, box

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class ContinuousRobotNavigationEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def is_goal_behind_wall(self, goal_position):
        # Check if goal_position is behind any wall
        for wall in self.maze_walls:
            x, y, width, height = wall
            if goal_position[0] >= x and goal_position[0] <= x + width \
                    and goal_position[1] >= y and goal_position[1] <= y + height:
                return True
        return False
    def __init__(self):
        super(ContinuousRobotNavigationEnv, self).__init__()
        self.grid_size = 10
        self.observation_space = spaces.Box(low=0, high=self.grid_size, shape=(2,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)  # Continuous action space

        self.velocity_scale = 0.3
        self.lidar_range = 3.0

        self.num_humans = 5  # Number of humans
        self.human_positions = [self._random_position() for _ in range(self.num_humans)]
        self.human_velocities = [self._random_velocity() for _ in range(self.num_humans)]
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]
        
        # self.num_obstacles = 3  # Number of static obstacles
        # self.obstacle_positions = [self._random_position() for _ in range(self.num_obstacles)]

        # self.maze_walls = [
        #     (5, 8, 0.3, 1.5),   # Example wall 1
        #     (0, 7, 1.5, 0.3),   # Example wall 2
        #     (6.5, 6.5, 1.5, 0.3),   # Example wall 3
        #     (4, 4.5, 0.3, 1.5),   # Example wall 4
        #     (1.5, 2, 0.3, 1.5), #5th wall
        #     (5, 2.5, 1.5, 0.3),  #6th wall
        #     (8, 4, 1.5, 0.3),    #7th wall
        #     (7, 0, 0.3, 1.5)   #8th wall    
        # ]

        self.maze_walls = [
            #(5, 8, 0.3, 1),   # Example wall 1
            # (0, 7, 1, 0.3),   # Example wall 2
            # (6.5, 6.5, 1, 0.3),   # Example wall 3
            # # (4, 4.5, 0.3, 1),   # Example wall 4
            # (1.5, 2, 0.3, 1), #5th wall
            # (5, 2.5, 1, 0.3),  #6th wall
            # (8, 4, 1, 0.3),    #7th wall
            # (7, 0, 0.3, 1),    #8th wall
            # (0, 0, self.grid_size, 0.3),               # Bottom boundary wall
            # (0, self.grid_size - 0.3, self.grid_size, 0.3),  # Top boundary wall
            # (0, 0, 0.3, self.grid_size),               # Left boundary wall
            # (self.grid_size - 0.3, 0, 0.3, self.grid_size)  # Right boundary wall#8th wall    
        ]
        
        self.goal_position = (0, 0)

        self.frames = []  # Initialize the frames list

        

        self.reset()

    def _random_position(self):
        return [np.random.uniform(0, self.grid_size - 1), np.random.uniform(0, self.grid_size - 1)]

    def _random_velocity(self):
        # return [np.random.uniform(-1, 1), np.random.uniform(-1, 1)]
        return [np.random.uniform(-1, 1) * self.velocity_scale, np.random.uniform(-1, 1) * self.velocity_scale]

    def reset(self):
        self.robot_position = [0, 0]
        self.robot_orientation = np.pi/4  #looking diagonally up
        self.human_positions = [self._random_position() for _ in range(self.num_humans)]
        self.human_velocities = [self._random_velocity() for _ in range(self.num_humans)]
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]
        # self.obstacle_positions = [self._random_position() for _ in range(self.num_obstacles)]
        self.state = np.array(self.robot_position, dtype=np.float32)
        # self.goal_position = self._random_position()
        # while True:
        #     self.goal_position = self._random_position()
        #     if not self.is_goal_behind_wall(self.goal_position):
        #         break
        self.goal_position = (6,4)
        self.frames = []  # Reset frames list at the start of each episode
        
        return self.state

    def check_wall_collision(self, position):
        for wall in self.maze_walls:
            x, y, width, height = wall
            if position[0] >= (x-0.1) and position[0] <= x + width and position[1] >= (y-0.1) and position[1] <= y + height:
                return True
        return False
        
    def predict_human_positions(self):
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]  # Reset predicted positions
        num_predictions = 5  # Number of future positions to predict
        
        for i in range(self.num_humans):
            future_positions = []
            future_position = np.array(self.human_positions[i])
            for _ in range(num_predictions):
                future_position += np.array(self.human_velocities[i])
                future_position[0] = np.clip(future_position[0], 0, self.grid_size - 1)
                future_position[1] = np.clip(future_position[1], 0, self.grid_size - 1)
                future_positions.append(future_position.copy())
        
            # Update current position
            self.human_positions[i][0] += self.human_velocities[i][0]
            self.human_positions[i][1] += self.human_velocities[i][1]
        
            # Check for boundary collisions and reverse velocity if necessary
            if self.human_positions[i][0] < 0 or self.human_positions[i][0] >= self.grid_size:
                self.human_velocities[i][0] *= -1
            if self.human_positions[i][1] < 0 or self.human_positions[i][1] >= self.grid_size:
                self.human_velocities[i][1] *= -1

            if self.check_wall_collision(self.human_positions[i]):
                self.human_velocities[i][0] *= -1
                self.human_velocities[i][1] *= -1
        
            # Update future positions if within lidar range
            if np.linalg.norm(np.array(self.robot_position) - np.array(self.human_positions[i])) <= self.lidar_range:
                self.predicted_human_positions[i] = future_positions
        


    def check_collision(self, position):
        # Check collision with humans
        for human_pos in self.human_positions:
            if np.linalg.norm(np.array(self.robot_position) - np.array(human_pos)) < 1.0:  # Collision threshold
                return True
        
        # Check collision with obstacles
        # for obstacle_pos in self.obstacle_positions:
        #     if np.linalg.norm(np.array(self.robot_position) - np.array(obstacle_pos)) < 1.0:  # Collision threshold
        #         return True

        for wall in self.maze_walls:
            x, y, width, height = wall
            if position[0] >= (x-0.3) and position[0] <= x + width and position[1] >= (y-0.3) and position[1] <= y + height:
                return True
        
        return False

        
    def step(self, action):
        self.state = np.array(self.robot_position, dtype=np.float32)
        # Apply scaling to the action to reduce velocity
        scaled_action = np.array(action) * self.velocity_scale
    
        # Calculate the new robot position
        new_robot_position = [
            self.robot_position[0] + scaled_action[0],
            self.robot_position[1] + scaled_action[1]
        ]
    
        # Clip the new position to stay within the environment bounds
        new_robot_position = [
            np.clip(new_robot_position[0], 0, self.grid_size - 1),
            np.clip(new_robot_position[1], 0, self.grid_size - 1)
        ]
    
        # Check if the new position collides with maze walls
        if not self.check_collision(new_robot_position):
            self.robot_position = new_robot_position
        else:
            done = True
            return self.state, -10, done, {}
    
        if np.linalg.norm(scaled_action) > 0:
            self.robot_orientation = np.arctan2(scaled_action[1], scaled_action[0])
    
        self.predict_human_positions()
        
        goal_threshold = 0.7
        distance_to_goal = np.linalg.norm(np.array(self.robot_position) - np.array(self.goal_position))

        reward = -distance_to_goal

        done = False

        if distance_to_goal < 0.7: 
            reward += 100  
            done = True
        else:
            collision = self.check_collision(self.robot_position)
            if collision:
                reward -= 10  
                done = True
            else:
                done = False
    
        # if done:
        #     self.reset()  
        
        self.state = np.array(self.robot_position, dtype=np.float32)
        # print(self.state)
        info = {}  # Add any additional info you want to pass
    
        return self.state, reward, done, info
    def render(self, mode='human'):
        plt.figure(figsize=(6, 6))
        ax = plt.gca()
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_aspect('equal')
    
        # Draw the goal position
        goal = plt.Circle(self.goal_position, 0.5, color='green')
        ax.add_artist(goal)
    
        # Draw the robot
        robot = plt.Circle(self.robot_position, 0.5, color='red')
        ax.add_artist(robot)
    
        # Draw the humans
        for human_pos in self.human_positions:
            human = plt.Circle(human_pos, 0.5, color='blue')
            ax.add_artist(human)

        
        #static obstacles
        # for obstacle_pos in self.obstacle_positions:
        #     obstacle = plt.Circle(obstacle_pos, 0.5, color='black')
        #     ax.add_artist(obstacle)

        #walls
        for wall in self.maze_walls:
            x, y, width, height = wall
            rect = plt.Rectangle((x, y), width, height, color='black')
            ax.add_patch(rect)
    
        
        #Orientation
        arrow_length = 0.7
        dx = arrow_length * np.cos(self.robot_orientation)
        dy = arrow_length * np.sin(self.robot_orientation)
        arrow = FancyArrow(self.robot_position[0], self.robot_position[1], dx, dy, color='black', width=0.1)
        ax.add_patch(arrow)

        self.render_lidar(ax)

        #Trajectory prediction
        for predicted_positions in self.predicted_human_positions:
            for future_pos in predicted_positions:
                future_circle = plt.Circle(future_pos, 0.2, color='pink', alpha=0.3)
                ax.add_artist(future_circle)

        
    
        plt.axis('off')
    
        # Save frame to list
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        frame = imageio.imread(buf)
        self.frames.append(frame)
        buf.close()
        plt.close() 

        # Show the plot in real-time
        # plt.imshow(frame)
        # plt.axis('off')
        # plt.pause(0.1)
        # plt.clf()

    def render_lidar(self, ax):
        lidar_segments = 36
        angle_increment = 2 * np.pi / lidar_segments
        for i in range(lidar_segments):
            angle = i * angle_increment
            end_point = self.robot_position + np.array([np.cos(angle), np.sin(angle)]) * self.lidar_range
            ray_end_point = self.get_ray_end_point(self.robot_position, end_point)
            ax.plot([self.robot_position[0], ray_end_point[0]], [self.robot_position[1], ray_end_point[1]], color='dimgray', alpha=0.2)

    def get_ray_end_point(self, start, end):
        for wall in self.maze_walls:
            x, y, width, height = wall
            if self.line_intersects_rect(start, end, x, y, width, height):
                return self.get_intersection_point(start, end, x, y, width, height)
        return end

    def line_intersects_rect(self, start, end, x, y, width, height):
        def ccw(A, B, C):
            return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

        A = start
        B = end
        C1 = [x, y]
        C2 = [x + width, y]
        C3 = [x, y + height]
        C4 = [x + width, y + height]
        return (ccw(A, C1, C2) != ccw(B, C1, C2) and ccw(A, B, C1) != ccw(A, B, C2)) or \
               (ccw(A, C3, C4) != ccw(B, C3, C4) and ccw(A, B, C3) != ccw(A, B, C4)) or \
               (ccw(A, C1, C3) != ccw(B, C1, C3) and ccw(A, B, C1) != ccw(A, B, C3)) or \
               (ccw(A, C2, C4) != ccw(B, C2, C4) and ccw(A, B, C2) != ccw(A, B, C4))

    def get_intersection_point(self, start, end, x, y, width, height):
        def line(p1, p2):
            A = (p1[1] - p2[1])
            B = (p2[0] - p1[0])
            C = (p1[0] * p2[1] - p2[0] * p1[1])
            return A, B, -C

        def intersection(L1, L2):
            D = L1[0] * L2[1] - L1[1] * L2[0]
            Dx = L1[2] * L2[1] - L1[1] * L2[2]
            Dy = L1[0] * L2[2] - L1[2] * L2[0]
            if D != 0:
                return Dx / D, Dy / D
            else:
                return False

        walls = [
            ([x, y], [x + width, y]),
            ([x, y], [x, y + height]),
            ([x + width, y], [x + width, y + height]),
            ([x, y + height], [x + width, y + height])
        ]
        closest_point = end
        min_distance = np.linalg.norm(np.array(start) - np.array(end))
        for wall in walls:
            L1 = line(start, end)
            L2 = line(wall[0], wall[1])
            R = intersection(L1, L2)
            if R:
                dist = np.linalg.norm(np.array(start) - np.array(R))
                if dist < min_distance:
                    min_distance = dist
                    closest_point = R
        return closest_point
    

In [ ]:
class AttentionModule(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AttentionModule, self).__init__()
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, hidden_dim)
    
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # Compute attention scores and weights
        attention_scores = torch.bmm(Q, K.transpose(1, 2)) / (x.size(-1) ** 0.5)
        attention_weights = F.softmax(attention_scores, dim=-1)
        
        # Compute attention output
        attention_output = torch.bmm(attention_weights, V)
        attention_output = self.fc_out(attention_output)
        
        return attention_output, attention_weights


In [ ]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128, num_humans=5):
        super(Actor, self).__init__()
        self.num_humans = num_humans
        self.hidden_size = hidden_size
        
        self.attention = AttentionModule(2, hidden_size)  # Each human has 2 dimensions (x, y)
        self.goal_attention = AttentionModule(2, hidden_size)  # Goal also has 2 dimensions (x, y)
        
        # The input size to fc1 is:
        # state_size + hidden_size (from humans) + hidden_size (from goal)
        self.fc1 = nn.Linear(state_size + hidden_size + hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
    
    def forward(self, state, humans, goal):
        # Reshape humans for attention module
        humans = humans.view(-1, self.num_humans, 2)  # Reshape to (batch_size, num_humans, 2)
        
        # Apply attention to humans
        human_attn_output, human_attn_weights = self.attention(humans)
        
        # Apply attention to goal
        goal = goal.unsqueeze(1)  # Add extra dimension for compatibility
        goal_attn_output, goal_attn_weights = self.goal_attention(goal)
        
        # Concatenate state with attention outputs
        x = torch.cat((state, human_attn_output.mean(1), goal_attn_output.squeeze(1)), dim=-1)
        
        # print(f"Shape of x before fc1: {x.shape}")  # Debugging line
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))  # Output action is in range [-1, 1]
        return x, human_attn_weights, goal_attn_weights

class Critic(nn.Module):
    def __init__(self, state_size, action_size, human_size, goal_size):
        super(Critic, self).__init__()
        
        self.human_attention = nn.Linear(human_size, human_size)  # Replace with actual attention mechanism
        self.goal_attention = nn.Linear(goal_size, goal_size)    # Replace with actual attention mechanism
        
        self.fc1 = nn.Linear(state_size + human_size + goal_size + action_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, state, action, humans, goal):
        # Compute attention outputs
        human_attn_output = self.human_attention(humans)
        goal_attn_output = self.goal_attention(goal)
        
        # Ensure outputs are tensors and handle the dimensions correctly
        if isinstance(human_attn_output, tuple):
            human_attn_output = human_attn_output[0]
        if isinstance(goal_attn_output, tuple):
            goal_attn_output = goal_attn_output[0]
        
        # Ensure the dimensions are compatible for concatenation
        state = state.flatten(start_dim=1)
        action = action.flatten(start_dim=1) if isinstance(action, torch.Tensor) else torch.tensor(action).flatten(start_dim=1)
        
        # Adjust attention outputs
        if len(human_attn_output.shape) == 3:
            human_attn_output = human_attn_output.mean(dim=1)
        if len(goal_attn_output.shape) == 3:
            goal_attn_output = goal_attn_output.squeeze(1)

        # Concatenate tensors
        x = torch.cat((state, human_attn_output, goal_attn_output, action), dim=-1)
        
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x




In [ ]:
class OUActionNoise:
    def __init__(self, mu, sigma, theta=0.15, dt=1e-2, initial_state=None):
        self.mu = mu
        self.sigma = sigma
        self.theta = theta
        self.dt = dt
        self.initial_state = initial_state
        self.reset()

    def reset(self):
        self.state = np.copy(self.initial_state) if self.initial_state is not None else np.zeros_like(self.mu)

    def __call__(self):
        x = self.state
        dx = self.theta * (self.mu - x) * self.dt + self.sigma * np.sqrt(self.dt) * np.random.randn(*self.mu.shape)
        self.state = x + dx
        return self.state

In [ ]:
class ReplayBuffer:
    def __init__(self, max_size, batch_size, state_size, action_size, human_size, goal_size):
        self.max_size = max_size
        self.batch_size = batch_size
        self.state_size = state_size
        self.action_size = action_size
        self.human_size = human_size
        self.goal_size = goal_size
        self.buffer = [None] * max_size
        self.idx = 0
        self.current_size = 0
        
    def add(self, state, action, reward, next_state, done, humans, next_humans, goal):
        experience = (state, action, reward, next_state, done, humans, next_humans, goal)
        self.buffer[self.idx] = experience
        self.idx = (self.idx + 1) % self.max_size
        self.current_size = min(self.current_size + 1, self.max_size)

        
    def sample(self, batch_size):
        batch = np.random.choice(self.current_size, batch_size, replace=False)
        states = np.zeros((batch_size, self.state_size))
        actions = np.zeros((batch_size, self.action_size))
        rewards = np.zeros(batch_size)
        next_states = np.zeros((batch_size, self.state_size))
        dones = np.zeros(batch_size)
        humans = np.zeros((batch_size, self.human_size))
        next_humans = np.zeros((batch_size, self.human_size))
        goals = np.zeros((batch_size, self.goal_size))
    
        for i, idx in enumerate(batch):
            states[i] = self.buffer[idx][0]
            actions[i] = self.buffer[idx][1]
            rewards[i] = self.buffer[idx][2]
            next_states[i] = self.buffer[idx][3]
            dones[i] = self.buffer[idx][4]
            humans[i] = self.buffer[idx][5]
            next_humans[i] = self.buffer[idx][6]
            goals[i] = self.buffer[idx][7]
    
        return states, actions, rewards, next_states, dones, humans, next_humans, goals
    
    def __len__(self):
        return self.current_size


In [ ]:
class DDPGAgentWithAttention:
    def __init__(self, state_size, action_size, human_dim, num_humans, goal_dim, buffer_size, batch_size, lr_actor, lr_critic, gamma, tau):
        self.actor = Actor(state_size, action_size, num_humans=num_humans)
        self.actor_target = Actor(state_size, action_size, num_humans=num_humans)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_actor)
        
        self.critic = Critic(state_size, action_size, human_size=human_dim * num_humans, goal_size=goal_dim)
        self.critic_target = Critic(state_size, action_size, human_size=human_dim * num_humans, goal_size=goal_dim)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_critic)
        
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.replay_buffer = ReplayBuffer(max_size=buffer_size, batch_size=batch_size,
                                          state_size=state_size, action_size=action_size,
                                          human_size=human_dim * num_humans, goal_size=goal_dim)

        # Initialize target networks with same weights as original networks
        self._update_targets(self.actor_target, self.actor)
        self._update_targets(self.critic_target, self.critic)
    
    def _update_targets(self, target_model, model):
        for target_param, param in zip(target_model.parameters(), model.parameters()):
            target_param.data.copy_(param.data)
            
    def act(self, state, humans, goal):
        state = torch.FloatTensor(state).unsqueeze(0)
        humans = torch.FloatTensor(humans).unsqueeze(0)
        goal = torch.FloatTensor(goal).unsqueeze(0)
        
        self.actor.eval()
        with torch.no_grad():
            action, _, _ = self.actor(state, humans, goal)
        self.actor.train()
        
        return action.squeeze(0).numpy()
        
    def remember(self, state, action, reward, next_state, done, humans, next_humans, goal):
        self.replay_buffer.add(state, action, reward, next_state, done, humans, next_humans, goal)

  
    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        
        # Sample from memory
        states, actions, rewards, next_states, dones, humans, next_humans, goals = self.replay_buffer.sample(self.batch_size)
        
        # Convert to tensors
        states = torch.tensor(states, dtype=torch.float).to(device)
        actions = torch.tensor(actions, dtype=torch.float).to(device)
        rewards = torch.tensor(rewards, dtype=torch.float).to(device)
        next_states = torch.tensor(next_states, dtype=torch.float).to(device)
        dones = torch.tensor(dones, dtype=torch.float).to(device)
        humans = torch.tensor(humans, dtype=torch.float).to(device)
        next_humans = torch.tensor(next_humans, dtype=torch.float).to(device)
        goals = torch.tensor(goals, dtype=torch.float).to(device)
    
        # Get next actions
        next_actions = self.actor_target(next_states, next_humans, goals)
        if isinstance(next_actions, tuple):
            next_actions = next_actions[0]
        
        # Compute Q targets
        Q_targets_next = self.critic_target(next_states, next_actions, next_humans, goals)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))
        
        # Compute Q expected
        Q_expected = self.critic(states, actions, humans, goals)
        # Q_targets = Q_targets.view_as(Q_expected)
        
        # Compute critic loss
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Compute actor loss
        actions_pred = self.actor(states, humans, goals)
        if isinstance(actions_pred, tuple):
            actions_pred = actions_pred[0]
        
        actor_loss = -self.critic(states, actions_pred, humans, goals).mean()
        
        # Optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Soft update the target networks
        self.soft_update(self.critic, self.critic_target)
        self.soft_update(self.actor, self.actor_target)

    def soft_update(self, local_model, target_model, tau=1e-3):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
    # def _soft_update_target(self, target_model, model, tau):
    #     for target_param, param in zip(target_model.parameters(), model.parameters()):
    #         target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)


In [ ]:
def main():
    env = ContinuousRobotNavigationEnv()
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    human_dim = 2  # Assuming each human is represented by (x, y) coordinates
    num_humans = env.num_humans
    goal_dim = 2  # Assuming goal is represented by (x, y) coordinates
    
    buffer_size = 10000
    batch_size = 64
    lr_actor = 1e-3
    lr_critic = 1e-3
    gamma = 0.99
    tau = 1e-3

    
    ddpg_agent = DDPGAgentWithAttention(state_dim, action_dim, human_dim, num_humans, goal_dim,
                                        buffer_size, batch_size, lr_actor, lr_critic, gamma, tau)
    replay_buffer = ReplayBuffer(max_size=buffer_size, batch_size=batch_size,
                                 state_size=state_dim, action_size=action_dim,
                                 human_size=human_dim * num_humans, goal_size=goal_dim)
    
    # Training loop
    num_episodes = 150
    episode_rewards = []
    
    for episode in range(num_episodes):
        state = env.reset()
        humans = np.array(env.human_positions).flatten()  # Flatten human positions
        goal = np.array(env.goal_position)
        episode_reward = 0
        env.frames = []  # Clear frames for the new episode
        
        for t in range(100):
            action = ddpg_agent.act(state, humans, goal)
            next_state, reward, done, _ = env.step(action)
            next_humans = np.array(env.human_positions).flatten()  # Update human positions
            
            ddpg_agent.remember(state, action, reward, next_state, done, humans, next_humans, goal)
            
            state = next_state
            humans = next_humans
            episode_reward += reward
            
            env.render()
            
            if done:
                break
        
        # Learn from the experiences in the replay buffer
        ddpg_agent.learn()
        episode_rewards.append(episode_reward)
        print(f"Episode {episode}, Reward: {episode_reward}")
        
        # Save GIF
        imageio.mimsave(f'episode_{episode}.gif', env.frames, fps=10)
    
    plt.plot(range(num_episodes), episode_rewards)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward')
    plt.title('Cumulative Reward over Episodes')
    plt.show()

if __name__ == "__main__":
    main()